<a href="https://colab.research.google.com/github/wolgank/INGESOFT/blob/main/AdD_trabajo_grupal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/usuario/repositorio.git

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

In [2]:
# ----------------------------
# CARGA DE CIFAR-10
# ----------------------------

# Transformaciones: convertir a escala de grises + tensor + normalización
transform = transforms.Compose([
transforms.Grayscale(num_output_channels=1),
transforms.ToTensor(),
transforms.Normalize((0.5,), (0.5,))
])

In [3]:
# Descargar dataset CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

100%|██████████| 170M/170M [00:01<00:00, 102MB/s]


In [4]:
# Dividir en train (40k) y validación (10k)
train_size = 40000
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Estammos usando: {device}')

Estammos usando: cpu


In [6]:
# ----------------------------
# ARQUITECTURA FINAL
# ----------------------------
# SU CÓDIGO AQUI...


# ====================== CNN Mejorada p/ MNIST (PyTorch) - Verbose & Debug ======================
import os, time, math
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# ---------- Modelo ----------
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=10, p_drop=0.15):
        super().__init__()
        self.stage1 = nn.Sequential(
            ConvBlock(1, 32), ConvBlock(32, 32),
            nn.MaxPool2d(2), nn.Dropout2d(p_drop),
        )
        self.stage2 = nn.Sequential(
            ConvBlock(32, 64), ConvBlock(64, 64),
            nn.MaxPool2d(2), nn.Dropout2d(p_drop),
        )
        self.stage3 = nn.Sequential(
            ConvBlock(64, 128), ConvBlock(128, 256),
            nn.Dropout2d(p_drop),
        )
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Linear(256, num_classes))
    def forward(self, x):
        return self.head(self.stage3(self.stage2(self.stage1(x))))

model = ImprovedCNN(num_classes=10, p_drop=0.15).to(device)

# ---------- Pérdida/Optimizador/Scheduler ----------
criterion = nn.CrossEntropyLoss(label_smoothing=0.10)
LR = 3e-3
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)
EPOCHS = 40
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=3e-5)

# ---------- AMP (API nueva) ----------
use_cuda = (device.type == "cuda")
autocast = torch.amp.autocast  # nueva API
scaler = torch.amp.GradScaler(device.type) if use_cuda else None  # en CPU no se usa

# ---------- Métricas & EarlyStopping ----------
@torch.no_grad()
def accuracy_from_logits(logits, y):
    return (logits.argmax(dim=1) == y).float().mean().item()

class EarlyStopping:
    def __init__(self, patience=10, ckpt_path="best_model.pt"):
        self.patience = patience; self.ckpt_path = ckpt_path
        self.best_metric = -float("inf"); self.wait = 0
    def step(self, metric, model):
        if metric > self.best_metric:
            self.best_metric = metric; self.wait = 0
            torch.save(model.state_dict(), self.ckpt_path)
            return False
        self.wait += 1
        return self.wait >= self.patience

early = EarlyStopping(patience=10, ckpt_path="best_model.pt")

# ---------- Smoke test: un batch ----------
xb0, yb0 = next(iter(train_loader))
print("Batch ejemplo:", tuple(xb0.shape), tuple(yb0.shape))
with torch.no_grad():
    xb0 = xb0.to(device).float()
    out0 = model(xb0[:4])
print("Logits ejemplo:", tuple(out0.shape))  # (4, num_classes)

# ---------- Entrenamiento con tqdm ----------
from tqdm.auto import tqdm

def run_epoch(loader, train_mode=True, show_pbar=True):
    model.train(mode=train_mode)
    total_loss = 0.0; total_acc = 0.0; n_batches = 0

    iterator = loader
    if show_pbar:
        iterator = tqdm(loader, leave=False)
        iterator.set_description("train" if train_mode else "valid")

    for xb, yb in iterator:
        xb = xb.to(device, non_blocking=True).float()
        yb = yb.to(device, non_blocking=True).long()

        if train_mode:
            optimizer.zero_grad(set_to_none=True)
            if use_cuda:
                with autocast(device_type="cuda"):
                    logits = model(xb); loss = criterion(logits, yb)
                scaler.scale(loss).backward()
                scaler.step(optimizer); scaler.update()
            else:
                logits = model(xb); loss = criterion(logits, yb)
                loss.backward(); optimizer.step()
        else:
            with torch.no_grad():
                logits = model(xb); loss = criterion(logits, yb)

        total_loss += loss.detach().item()
        total_acc  += accuracy_from_logits(logits, yb)
        n_batches  += 1

        if show_pbar:
            iterator.set_postfix(loss=f"{total_loss/n_batches:.4f}",
                                 acc=f"{total_acc/n_batches:.4f}")

    return total_loss / n_batches, total_acc / n_batches

history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]}
t0 = time.time()
for epoch in range(1, EPOCHS+1):
    print(f"\nEpoch {epoch}/{EPOCHS} — lr={scheduler.get_last_lr()[0]:.2e}", flush=True)
    train_loss, train_acc = run_epoch(train_loader, train_mode=True,  show_pbar=True)
    val_loss,   val_acc   = run_epoch(val_loader,   train_mode=False, show_pbar=True)
    scheduler.step()

    history["train_loss"].append(train_loss); history["val_loss"].append(val_loss)
    history["train_acc"].append(train_acc);   history["val_acc"].append(val_acc)

    print(f"  -> train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"val: loss={val_loss:.4f}, acc={val_acc:.4f}", flush=True)

    if early.step(val_acc, model):
        print(f"Early stopping en epoch {epoch}. Mejor val_acc={early.best_metric:.4f}")
        break

print(f"\nEntrenamiento terminado en {time.time()-t0:.1f}s. Mejor val_acc={early.best_metric:.4f}")

# ---------- Evaluación en test ----------
model.load_state_dict(torch.load("best_model.pt", map_location=device))
model.eval()
test_loss, test_acc = run_epoch(test_loader, train_mode=False, show_pbar=False)
print(f"\n==> Test: loss={test_loss:.4f} | acc={test_acc:.4f}")

# ---------- (Opcional) Curvas ----------
try:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].plot(history["train_loss"], label="train"); ax[0].plot(history["val_loss"], label="val")
    ax[0].set_title("Loss"); ax[0].legend()
    ax[1].plot(history["train_acc"],  label="train"); ax[1].plot(history["val_acc"],  label="val")
    ax[1].set_title("Accuracy"); ax[1].legend()
    plt.show()
except Exception as e:
    print("No se pudo graficar:", e)


Batch ejemplo: (128, 1, 32, 32) (128,)
Logits ejemplo: (4, 10)

Epoch 1/40 — lr=3.00e-03


  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  -> train: loss=1.7457, acc=0.4206 | val: loss=1.4602, acc=0.5747

Epoch 2/40 — lr=3.00e-03


  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

  -> train: loss=1.4481, acc=0.5771 | val: loss=1.3154, acc=0.6414

Epoch 3/40 — lr=2.98e-03


  0%|          | 0/313 [00:00<?, ?it/s]

KeyboardInterrupt: 